In [3]:
import pandas as pd
import numpy as np
import nb_black
import os
import warnings

warnings.filterwarnings("ignore")

%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


## Métodos

Essa secao ficara todos os metodos e funçoes que serao utilizadas no projeto, a fim de deixar mais organizado e limpo para facilitar o entendimento

In [4]:
def print_shape_dfs(folder_path):
    files = os.listdir(folder_path)
    csv_files = [f for f in files if f.endswith(".csv")]

    for file in csv_files:
        filepath = os.path.join(folder_path, file)
        df = pd.read_csv(filepath, delimiter=";")
        print(f"{file} -> {df.shape}")

In [5]:
def concat_csv_files(folder_path):
    files = os.listdir(folder_path)
    csv_files = [f for f in files if f.endswith(".csv")]
    print_shape_dfs(folder_path)
    # Criar uma lista de dfs
    dfs = []
    for file in csv_files:
        filepath = os.path.join(folder_path, file)
        temp_df = pd.read_csv(filepath, delimiter=";")
        dfs.append(temp_df)

    # Adicionar o df do kaggle na lista de dfs do webscraping
    df_kaggle = pd.read_csv("dataZAP.csv", delimiter=";")
    dfs.append(df_kaggle)

    # Obter todas as colunas presentes em todos os dfs
    all_columns = set().union(*[set(df.columns) for df in dfs])

    # Reindexar as colunas em todos os dfs para garantir que eles tenham as mesmas colunas e na mesma ordem
    dfs = [df.reindex(columns=all_columns) for df in dfs]

    result = pd.concat(dfs, ignore_index=True)
    output_file = os.path.join("./", "dataZap_concatenated.csv")
    result.to_csv(output_file, sep=";", index=False)
    print(f"Shape do df inicial: {df_kaggle.shape}\nShape do df final: {result.shape}")

## Webscraping

### Pre-Incrementação

Essa análise abaixo foi necessária para que nosso script de webscraping tenha as mesmas colunas e formato do dataset que está no kaggle, então abaixo foi feito uma análise sobre o shape deles e também sobre quais colunas tem em um e que não tem no outro, até que se chegou no ponto final, que é quando as duas tem o mesmo shape e colunas iguais, que é o output dessa parte.

Isso impactou no código scrap_zap_v3 (nossa versão), pois tive que modificar as colunas que são filtradas e consequentemente criadas o dataframe.

dfc = dataset do kaggle

dfx = um dos datasets que o webscraping cria

In [6]:
df = pd.read_csv("dataZAP.csv", delimiter=";")
df.shape

(35772, 78)

In [7]:
df_aux = pd.read_csv(
    "webscraping-datasets/dataZap_ce_aluguel_casas_TODOS_3.csv", delimiter=";"
)
df_aux.shape

(352, 78)

In [8]:
dfc = []
for i in df.columns:
    dfc.append(i)

dfx = []
for i in df_aux.columns:
    dfx.append(i)

dfc = set(dfc)
dfx = set(dfx)

In [9]:
len(dfc.intersection(dfx))

78

In [10]:
list(dfc - dfx)

[]

In [11]:
list(dfx - dfc)

[]

Conclusao: Dessa forma final, todas os dataframes que forem realizados pelo webscraping terao as mesmas colunas e quantidades que o dataset do kaggle, facilitando a incrementacao dos dados via scraping.

### Utilização do Script de WebScraping

In [1]:
!python ./webscraping/scrap_zap_v3.py

TODOS -> TODOS -> es -> casas -> 3

Criando o arquivo dataZAP_aluguel_casas_TODOS.csv com os dados

Arquivo criado

Webscraping finalizado!


### Incrementação do Dataset

In [26]:
concat_csv_files("webscraping-datasets")

dataZap_ba_aluguel_casas_TODOS_3.csv -> (495, 78)
dataZap_ce_aluguel_casas_TODOS_3.csv -> (352, 78)
dataZap_es_aluguel_casas_TODOS_3.csv -> (106, 78)
dataZap_mg_aluguel_casas_TODOS_3.csv -> (1727, 78)
dataZap_rj_aluguel_casas_TODOS_3.csv -> (1227, 78)
dataZap_sp_aluguel_casas_TODOS_3.csv -> (8900, 78)
Shape do df inicial: (35772, 78)
Shape do df final: (48579, 78)


## Data Science

### Pre-processamento

In [113]:
df = pd.read_csv("dataZap_concatenated.csv", delimiter=";")

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48579 entries, 0 to 48578
Data columns (total 78 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   listing.address.neighborhood          48517 non-null  object 
 1   listing.mountainview                  48579 non-null  bool   
 2   listing.sauna                         48579 non-null  bool   
 3   listing.externalId                    48579 non-null  object 
 4   listing.pricingInfo.businessType      48579 non-null  object 
 5   listing.pricingInfo.monthlyCondoFee   36938 non-null  object 
 6   listing.pricingInfo.isRent            48579 non-null  bool   
 7   listing.address.level                 48579 non-null  object 
 8   listing.bathrooms                     48579 non-null  object 
 9   listing.displayAddressType            48579 non-null  object 
 10  listing.pricingInfo.isSale            48579 non-null  bool   
 11  listing.preview

In [14]:
df.describe()

,listing.pricingInfo.rentalPrice,listing.bedrooms,listing.unitsOnTheFloor,listing.unitFloor,listing.id
count,48579.000000,48579.000000,48579.00000,48579.000000,4.857900e+04
mean,100.198969,2.509665,0.24418,0.125877,2.477750e+09
std,249.669864,1.133065,1.64980,1.085948,2.175372e+08
min,1.000000,0.000000,0.00000,0.000000,4.535233e+07
25%,2.100000,1.000000,0.00000,0.000000,2.473924e+09
50%,3.750000,3.000000,0.00000,0.000000,2.486654e+09
75%,9.000000,3.000000,0.00000,0.000000,2.547760e+09
max,999.000000,14.000000,52.00000,85.000000,2.630391e+09


In [16]:
df.head()

,listing.address.neighborhood,listing.mountainview,listing.sauna,listing.externalId,listing.pricingInfo.businessType,listing.pricingInfo.monthlyCondoFee,listing.pricingInfo.isRent,listing.address.level,listing.bathrooms,listing.displayAddressType,...,listing.address.point.lat,imvl_type,account.licenseNumber,listing.suites,listing.bathtub,listing.pool,listing.pricingInfo.price,listing.address.country,listing.id,listing.pricingInfo.rentalTotalPrice
0,Guarajuba Monte Gordo,False,False,CO562A,RENTAL,700.0,True,CITY,5,STREET,...,NaN,casas,NaN,3.0,False,False,3.000,BR,2552533354,NaN
1,Nazaré,False,False,CA0218,RENTAL,NaN,True,STREET,3,NEIGHBORHOOD,...,NaN,casas,NaN,1.0,False,False,3.000,BR,2628408803,NaN
2,Pituba,False,False,741,RENTAL_SALE,NaN,True,STREET,3,ALL,...,-12.992829,casas,14268-J-BA,2.0,False,False,7.500,BR,2616700504,NaN
3,Gleba A,False,False,JC C232C,RENTAL,NaN,True,STREET,2,ALL,...,-12.69282,casas,NaN,1.0,False,False,1.800,BR,2629969856,NaN
4,Gleba A,False,False,JC 9E32C,RENTAL,NaN,True,STREET,2,ALL,...,-12.685308,casas,NaN,1.0,False,False,1.800,BR,2629953061,NaN


In [15]:
df.isna().sum()

listing.address.neighborhood               62
listing.mountainview                        0
listing.sauna                               0
listing.externalId                          0
listing.pricingInfo.businessType            0
                                        ...  
listing.pool                                0
listing.pricingInfo.price                   0
listing.address.country                     0
listing.id                                  0
listing.pricingInfo.rentalTotalPrice    11658
Length: 78, dtype: int64

#### Analisando as colunas

In [18]:
columns = [
    "listing.pricingInfo.isRent",
    "listing.pricingInfo.isSale",
    "listing.pricingInfo.price",
    "listing.pricingInfo.rentalPrice",
    "listing.pricingInfo.rentalTotalPrice",
    "listing.pricingInfo.salePrice",
]
df[columns]

,listing.pricingInfo.isRent,listing.pricingInfo.isSale,listing.pricingInfo.price,listing.pricingInfo.rentalPrice,listing.pricingInfo.rentalTotalPrice,listing.pricingInfo.salePrice
0,True,False,3.000,3.00,NaN,NaN
1,True,False,3.000,3.00,NaN,NaN
2,True,True,7.500,7.50,NaN,1.300.000
3,True,False,1.800,1.80,NaN,NaN
4,True,False,1.800,1.80,NaN,NaN
...,...,...,...,...,...,...
48574,True,False,1.8,1.80,2.720,normal
48575,True,False,1.19,1.19,1.434,normal
48576,True,False,940.0,940.00,1.054,normal
48577,True,False,1.6,1.60,2.294,normal


In [21]:
df["listing.pricingInfo.rentalTotalPrice"].isna().sum()

11658

In [22]:
df["listing.pricingInfo.price"].isna().sum()

0

In [20]:
df[(df["listing.pricingInfo.isRent"] == True)]

,listing.address.neighborhood,listing.mountainview,listing.sauna,listing.externalId,listing.pricingInfo.businessType,listing.pricingInfo.monthlyCondoFee,listing.pricingInfo.isRent,listing.address.level,listing.bathrooms,listing.displayAddressType,...,listing.address.point.lat,imvl_type,account.licenseNumber,listing.suites,listing.bathtub,listing.pool,listing.pricingInfo.price,listing.address.country,listing.id,listing.pricingInfo.rentalTotalPrice
0,Guarajuba Monte Gordo,False,False,CO562A,RENTAL,700.0,True,CITY,5,STREET,...,NaN,casas,NaN,3.0,False,False,3.000,BR,2552533354,NaN
1,Nazaré,False,False,CA0218,RENTAL,NaN,True,STREET,3,NEIGHBORHOOD,...,NaN,casas,NaN,1.0,False,False,3.000,BR,2628408803,NaN
2,Pituba,False,False,741,RENTAL_SALE,NaN,True,STREET,3,ALL,...,-12.992829,casas,14268-J-BA,2.0,False,False,7.500,BR,2616700504,NaN
3,Gleba A,False,False,JC C232C,RENTAL,NaN,True,STREET,2,ALL,...,-12.69282,casas,NaN,1.0,False,False,1.800,BR,2629969856,NaN
4,Gleba A,False,False,JC 9E32C,RENTAL,NaN,True,STREET,2,ALL,...,-12.685308,casas,NaN,1.0,False,False,1.800,BR,2629953061,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48574,Recreio Dos Bandeirantes,False,True,893106746,RENTAL,920,True,NEIGHBORHOOD,2,ALL,...,-23.017479,apartamentos,24344-J-SP,1,False,True,1.8,BR,2489855572,2.720
48575,Jardim Íris,False,False,893081971,RENTAL,244,True,STREET,1,ALL,...,-23.500983,apartamentos,24344-J-SP,0,False,False,1.19,BR,2487454665,1.434
48576,Canindé,False,False,893103422,RENTAL,114,True,STREET,1,ALL,...,-23.523386,apartamentos,24344-J-SP,0,False,False,940.0,BR,2489479893,1.054
48577,Cambuí,False,False,893107326,RENTAL,694,True,STREET,2,ALL,...,-22.902013,apartamentos,24344-J-SP,1,False,False,1.6,BR,2490025990,2.294


In [67]:
df["listing.pricingInfo.businessType"].value_counts()

RENTAL         36185
RENTAL_SALE    11535
SALE_RENTAL      859
Name: listing.pricingInfo.businessType, dtype: int64

In [68]:
df["listing.displayAddressType"].value_counts()

ALL             28658
STREET          12306
NEIGHBORHOOD     7615
Name: listing.displayAddressType, dtype: int64

In [84]:
df["listing.isInactive"].value_counts()

False    48020
True       559
Name: listing.isInactive, dtype: int64

In [86]:
# Tirando as colunas que nao importam para nossa analise

columns = [
    "listing.address.streetNumber",
    "Page",
    "account.legacyVivarealId",
    "account.legacyZapId",
    "listing.address.geoJson",
    "account.licenseNumber",
    "listing.address.ibgeCityId",
    "listing.id",
    "listing.legacyId",
    "listing.pricingInfo.priceVariation",
    "listing.description",
    "listing.images",
    "listing.videos",
    "listing.pricingInfo.isRent",
    "listing.pricingInfo.isSale",
    "listing.pricingInfo.price",
    "listing.pricingInfo.rentalTotalPrice",
    "listing.pricingInfo.salePrice",
    "listing.unitSubTypes",
    "listing.amenities",
    "account.logoUrl",
    "account.name",
    "link.href",
    "listing.updatedAt",
    "listing.subtitle",
    "listing.title",
    "listing.portal",
    "listing.createdAt",
    "listing.externalId",
    "listing.advertiserId",
    "listing.address.zipCode",
    "listing.address.street",
    "listing.address.point.lon",
    "listing.address.point.lat",
    "listing.address.precision",
    "listing.videoTour",
    "listing.address.point.source",
    "listing.address.country",
    "listing.address.confidence",
    "listing.publicationType",
    "listing.pricingInfo.businessType",
    "listing.address.state",
    "listing.listingType",
    "listing.link",
    "listing.propertyType",
    "listing.pricingInfo.businessLabel",
    "listing.isInactive",
]

lista_columns = []
for df_columns in df.columns:
    lista_columns.append(df_columns)

for column in columns:
    if column in lista_columns:
        df.drop(columns=column, axis=1, inplace=True)

In [87]:
df.shape

(48579, 42)

#### Limitando a regiao de interesse

Consideramos que a regiao importa para o calculo do modelo de previsao. Entao para isso, iremos fazer um filtro para que a nossa previsao seja mais fidedigna possivel com os valores apresentados. Dessa forma, vamos fazer a analise do preço de aluguel para o estado do Rio de Janeiro

In [114]:
pd.options.display.max_columns = 99
column_filter = df["listing.address.city"] == "Rio de Janeiro"
df = df[column_filter]

In [91]:
df.isna().sum()

listing.address.neighborhood             0
listing.mountainview                     0
listing.sauna                            0
listing.pricingInfo.monthlyCondoFee    326
listing.address.level                    0
listing.bathrooms                        0
listing.displayAddressType               0
listing.preview                          0
listing.totalAreas                      40
listing.sportcourt                       0
listing.unitTypes                        0
listing.pricingInfo.rentalPrice          0
listing.hottub                           0
listing.bedrooms                         0
listing.pricingInfo.period               0
listing.fireplace                        0
listing.guestpark                        0
listing.soundproofing                    0
listing.gym                              0
listing.floors                         378
listing.playground                       0
listing.partyhall                        0
listing.unitsOnTheFloor                  0
listing.bac

In [75]:
df_new.head(1)

,listing.address.neighborhood,listing.mountainview,listing.sauna,listing.pricingInfo.businessType,listing.pricingInfo.monthlyCondoFee,listing.address.level,listing.bathrooms,listing.displayAddressType,listing.preview,listing.totalAreas,listing.sportcourt,listing.address.state,listing.unitTypes,listing.hottub,listing.bedrooms,listing.pricingInfo.period,listing.fireplace,listing.guestpark,listing.soundproofing,listing.gym,listing.floors,listing.playground,listing.partyhall,listing.listingType,listing.unitsOnTheFloor,listing.backyard,listing.usageTypes,listing.propertyType,listing.pricingInfo.businessLabel,listing.garden,listing.link,listing.barbgrill,listing.acceptExchange,listing.usableAreas,listing.furnished,listing.pricingInfo.yearlyIptu,listing.unitFloor,listing.businessTypeContext,listing.tenniscourt,listing.parkingSpaces,listing.address.zone,type,listing.address.city,listing.publicationType,listing.isInactive,imvl_type,listing.suites,listing.bathtub,listing.pool
2689,Abolição,False,False,RENTAL,NaN,STREET,1,ALL,False,NaN,False,Rio de Janeiro,HOME,False,3,mês,False,False,False,False,NaN,False,False,USED,0,False,RESIDENTIAL,UNIT,Alugar,False,/imovel/aluguel-casa-3-quartos-abolicao-zona-n...,False,False,65,False,34.0,2,RENTAL,False,0.0,Zona Norte,NaN,Rio de Janeiro,STANDARD,False,casas,NaN,False,False


#### Completando (Fill) os valores nulos

In [98]:
# Como nao pode ter casa sem piso, sera transformado em 1 quando nulo
df["listing.floors"] = df["listing.floors"].fillna(1)

# Valor de iptu nula, deve ser sem iptu
df["listing.pricingInfo.yearlyIptu"] = df["listing.pricingInfo.yearlyIptu"].fillna(0)

# Vaga de estacionamento nula, deve ser sem vagas
df["listing.parkingSpaces"] = df["listing.parkingSpaces"].fillna(0)

# Area total nula, vamos admitir que é o mesmo tamanho da area usavel
df["listing.totalAreas"] = df["listing.totalAreas"].fillna(df["listing.usableAreas"])

In [99]:
df.isna().sum()

listing.address.neighborhood             0
listing.mountainview                     0
listing.sauna                            0
listing.pricingInfo.monthlyCondoFee    326
listing.address.level                    0
listing.bathrooms                        0
listing.displayAddressType               0
listing.preview                          0
listing.totalAreas                       0
listing.sportcourt                       0
listing.unitTypes                        0
listing.pricingInfo.rentalPrice          0
listing.hottub                           0
listing.bedrooms                         0
listing.pricingInfo.period               0
listing.fireplace                        0
listing.guestpark                        0
listing.soundproofing                    0
listing.gym                              0
listing.floors                           0
listing.playground                       0
listing.partyhall                        0
listing.unitsOnTheFloor                  0
listing.bac

In [117]:
df[df["listing.suites"].isna() == True][
    ["listing.bathrooms", "listing.bedrooms", "listing.suites"]
]

,listing.bathrooms,listing.bedrooms,listing.suites
2689,1,3,NaN
2695,1,3,NaN
2715,1,3,NaN
2740,2,3,NaN
2772,2,3,NaN
2815,2,3,NaN
2831,1,3,NaN
2885,3,3,NaN
2899,2,3,NaN
2951,2,3,NaN


In [118]:
df[["listing.bathrooms", "listing.bedrooms", "listing.suites"]]

,listing.bathrooms,listing.bedrooms,listing.suites
2689,1,3,NaN
2692,3,3,1.0
2693,1,3,1.0
2695,1,3,NaN
2701,2,3,1.0
...,...,...,...
48560,1,2,0
48566,2,3,1
48571,3,3,1
48572,2,1,0


In [119]:
df.drop("listing.suites", axis=1, inplace=True)

In [112]:
df.head(1)

,listing.address.neighborhood,listing.mountainview,listing.sauna,listing.pricingInfo.monthlyCondoFee,listing.address.level,listing.bathrooms,listing.displayAddressType,listing.preview,listing.totalAreas,listing.sportcourt,listing.unitTypes,listing.pricingInfo.rentalPrice,listing.hottub,listing.bedrooms,listing.pricingInfo.period,listing.fireplace,listing.guestpark,listing.soundproofing,listing.gym,listing.floors,listing.playground,listing.partyhall,listing.unitsOnTheFloor,listing.backyard,listing.usageTypes,listing.garden,listing.barbgrill,listing.acceptExchange,listing.usableAreas,listing.furnished,listing.pricingInfo.yearlyIptu,listing.unitFloor,listing.businessTypeContext,listing.tenniscourt,listing.parkingSpaces,listing.address.zone,type,listing.address.city,imvl_type,listing.bathtub,listing.pool
2689,Abolição,False,False,NaN,STREET,1,ALL,False,65,False,HOME,1.35,False,3,mês,False,False,False,False,1,False,False,0,False,RESIDENTIAL,False,False,False,65,False,34.0,2,RENTAL,False,0.0,Zona Norte,NaN,Rio de Janeiro,casas,False,False


#### Tirando outlier do rentalprice